In [1]:
import pathlib
from datetime import datetime
import pickle
from IPython.core.display import HTML, display
import re

import pandas as pd
import pymc3 as pm
from scipy.stats import mode
import numpy as np
from matplotlib import rcParams

In [2]:
%matplotlib inline

In [3]:
display(HTML("<style>.container {width: 90%}</style>"))

In [4]:
def convert_secs2dt(sec):
    """
    Converts seconds to python datetime object.
    :param sec 
    :return: datetime object
    """
    zd00 = datetime(2000, 1, 1)
    zd70 = datetime(1970, 1, 1)
    offset = (zd00 - zd70).total_seconds()
    z = datetime.utcfromtimestamp(sec + offset)
    return z


def get_doy(secs):
    """
    Converts seconds to fractional day of year.
    :param secs
    :return: fractional day of year 
    """
    z = convert_secs2dt(secs)
    y = z.year
    return (secs + (datetime(2000, 1, 1) - datetime(y, 1, 1)).total_seconds()) / 86400


def load_format_data(filepath, minimal=True, columns=None, quantity='chl_a'):
    """
    Loads chlorophyll data into a pandas dataframe,
    formats time entries, and creates a datetime index.
    :param filepath: string or pathlib object  
    :param minimal: if True returns only chl_a_mean; drops the rest.
    :return: pandas datetime indexed dataframe
    """
    if columns is None:
        columns = ['time', 'nbins', 'mean', 'median', 'stdv']
    
    df = pd.read_csv(filepath, delim_whitespace=True, names=columns)
    df['datetime'] = df.time.apply(convert_secs2dt)
    df.set_index('datetime', inplace=True)
    if minimal:
        df = df[['mean']]
    df.rename(columns={'mean': '%s_mean' % quantity}, inplace=True)
    return df


def compute_phyto_C_stats(df_):
    bbp_443_ = df_.bbp_443_giop_adj_mean.values.reshape(1,-1)
    phyto_c_trace_ = intercept + bbp_443_ * slope
    phyto_c_mean = phyto_c_trace_.mean(axis=0)
    phyto_c_hpd = pm.hpd(phyto_c_trace_)
    df_.insert(loc=1, column='phyto_C_2.5%', value=phyto_c_hpd[:, 0])
    df_.insert(loc=2, column='phyto_C_97.5%', value=phyto_c_hpd[:, 1])
    df_.insert(loc=2, column='phyto_C_mean', value=phyto_c_mean)
    return df_

def get_monthly_means(df, **kwargs):
    """
    Groups data by month and compute annual cycle based on monthly means.
    :param df: 
        datetime indexed pandas dataframe
    :param kwargs:
        year_start (optional): string, slice start
        year_end (optional): string, slice end
    :return: 
        month-indexed pandas dataframe with monthly means
    """
    year_start = kwargs.pop('year_start', df.index.year[0])
    year_end = kwargs.pop('year_end', df.index.year[-1])
    return df.loc[str(year_start): str(year_end)].groupby(lambda x: x.month).aggregate('mean')


def get_anomaly(df, df_ann_cycle, name='chl_a_mean', anomaly_name='anomaly'):
    """
    Computes annomaly by removing monthly mean for a given month
    :param df:
        pandas dataframe with [name] parameter column
    :param df_ann_cycle:
        pandas dataframe of length 12 containing monthly means
    :param name:
        str, label of quantity to get anomaly from
    :return:
        None
    """

    for month in df_ann_cycle.index:
        idx = df.index.month == month
        df.loc[idx, anomaly_name] = df.loc[idx, name] - df_ann_cycle.loc[month, name]


def test(datadir):
    testfile = datadir / 'ar2018.0m_AtlN55_chlor_a.txt'
    df_test = load_format_data(testfile, minimal=False)
    t0 = df_test.time[0]    
    z0 = convert_secs2dt(t0)
    zstr0 = z0.strftime('%Y%j%H%M%S')
    try:
        assert zstr0 + '000' == str(2002197194740000)
    except AssertionError as e:
        print(e)
        print(zstr0)
    assert z0.year == 2002
    doy0 = get_doy(t0)
    assert doy0 == 196.82476851851851
    tl = df_test.tail(1).time.values
    zl = convert_secs2dt(tl)
    zstrl = zl.strftime('%Y%j%H%M%S')
    assert zstrl + '000' == str(2016321013320000)
    assert zl.year == 2016
    doyl = get_doy(tl)
    assert doyl == 320.06481481481484
    print("all tests passed")

def get_sensor(row, prod):
    if np.isfinite(row[f'adj_{prod}_s'] * row[f'adj_{prod}_a']):
        return 'both'
    else:
        if np.isfinite(row[f'adj_{prod}_s']):
            return 'swf'
        else:
            return 'aqua'

In [6]:
path=pathlib.Path('/accounts/ekarakoy/DEV-ALL/State_of_the_Climate/soc2018/TIMESERIES/pso_test/')

In [7]:
with open('./pklJar/pooled_params.pkl', 'rb') as fb:
    pooled_params_dict = pickle.load(fb)
slope = pooled_params_dict['slope']
slope = slope.reshape(-1, 1)
intercept = pooled_params_dict['intercept']
intercept = intercept.reshape(-1, 1)

In [8]:
for file in path.glob('*.txt'):
    print(file.as_posix())

/accounts/ekarakoy/DEV-ALL/State_of_the_Climate/soc2018/TIMESERIES/pso_test/st134m_eqpso_bbp_443_giop.txt
/accounts/ekarakoy/DEV-ALL/State_of_the_Climate/soc2018/TIMESERIES/pso_test/at168m_eqpso_bbp_443_giop.txt
/accounts/ekarakoy/DEV-ALL/State_of_the_Climate/soc2018/TIMESERIES/pso_test/st134m_nhpso_bbp_443_giop.txt
/accounts/ekarakoy/DEV-ALL/State_of_the_Climate/soc2018/TIMESERIES/pso_test/st134m_shpso_bbp_443_giop.txt
/accounts/ekarakoy/DEV-ALL/State_of_the_Climate/soc2018/TIMESERIES/pso_test/at168m_shpso_bbp_443_giop.txt
/accounts/ekarakoy/DEV-ALL/State_of_the_Climate/soc2018/TIMESERIES/pso_test/at168m_nhpso_bbp_443_giop.txt


In [9]:
def load_regionals(path_, **kwargs):
    regions = kwargs.pop('regions', ['eqpso', 'nhpso', 'shpso'])
    prods = kwargs.pop('prods', ['chlor_a', 'bbp_443_giop'])
    sensors = kwargs.pop('sensors', ['aqua' ,'swf', 'swf_aqua'])
    dict_ = {reg: {prod: dict.fromkeys(sensors) 
               for prod in prods} 
         for reg in regions}
    for file in path_.glob('*.txt'):
    
        key_reg, key_prod = None, None
        for reg in regions:
            if reg in file.name:
                key_reg = reg
                break
        for prod in prods:
            if prod in file.name:
                key_prod=prod
                break
        sensor = 'aqua' if 'ar' in file.name or 'at' in file.name else 'swf'
        df_ = load_format_data(file, quantity=key_prod)
        df_.rename({col: f'{sensor[0]}_{col}' for col in df_.columns},
                   axis=1, inplace=True)
        if sensor == 'swf':
            df_ = df_.loc[:'2007', :]
        dict_[key_reg][key_prod][sensor] = df_
    return dict_

In [26]:
regions = ['eqpso', 'nhpso', 'shpso']
prods=['bbp_443_giop']

In [10]:
dct_df = load_regionals(path, prods=['bbp_443_giop'])

In [14]:
def consolidate(dct_df_, **kwargs):

    regions = kwargs.pop('regions', ['eqpso', 'nhpso', 'shpso'])
    prods = kwargs.pop('prods', ['chlor_a', 'bbp_443_giop'])
    for reg in regions:
        for prod in prods:
            df_aqua = dct_df_[reg][prod]['aqua']
            df_swf = dct_df_[reg][prod]['swf']
            df_all = pd.concat((df_aqua.resample('MS', loffset=pd.Timedelta(14, 'd')).first(),
                                       df_swf.resample('MS', loffset=pd.Timedelta(14, 'd')).first()),
                                      axis=1)
            df_both=df_all.dropna().copy()
            df_both.insert(0, f'{prod}_mean', df_both[[f's_{prod}_mean',
                                                       f'a_{prod}_mean']
                                                     ].mean(axis=1))
            df_both.insert(1, f'aqua-{prod}_mean',
                           df_both[f'a_{prod}_mean']- df_both[f'{prod}_mean'])
            df_both.insert(1, f'{prod}_mean-swf',
                           df_both[f'{prod}_mean'] - df_both[f's_{prod}_mean'])
            df_all[f'adj_{prod}_s'] = df_all[f's_{prod}_mean'] + df_both[f'{prod}_mean-swf'].mean()
            df_all[f'adj_{prod}_a'] = df_all[f'a_{prod}_mean'] + df_both[f'aqua-{prod}_mean'].mean()
            df_all[f'{prod}_adj_mean'] = df_all[[f'adj_{prod}_s', f'adj_{prod}_a']].mean(axis=1)
            df_all_sub = df_all[[f'{prod}_adj_mean']].copy()
            if 'bbp' in prod:
                df_all_sub = compute_phyto_C_stats(df_all_sub)
            prod_ = 'phyto_C' if 'bbp' in prod else f'{prod}_adj'
            prod_ann_cycle = get_monthly_means(df_all_sub[[f'{prod_}_mean']],
                                               year_start=2003, year_end=2011)
            get_anomaly(df_all_sub, prod_ann_cycle, name=f'{prod_}_mean',
                       anomaly_name=f'{prod_}_anomaly')
            df_all_sub['sensor'] = df_all.apply(get_sensor, args=(prod,), axis=1)
            dct_df_[reg][prod].update(dict(df_all=df_all_sub))
    return dct_df_

In [16]:
dct_df = consolidate(dct_df, prods=['bbp_443_giop'])

In [17]:
reg = 'eqpso'
dct_df[reg]['bbp_443_giop']['df_all'].head()

,bbp_443_giop_adj_mean,phyto_C_2.5%,phyto_C_mean,phyto_C_97.5%,phyto_C_anomaly,sensor
datetime,,,,,,
1997-11-15,0.002056,22.581171,24.636688,26.757878,0.351013,swf
1997-12-15,0.001952,21.562679,23.416629,25.457866,-0.148330,swf
1998-01-15,0.001897,21.003003,22.765257,24.761081,-0.102792,swf
1998-02-15,0.001886,20.887363,22.639052,24.620916,-0.255566,swf
1998-03-15,0.001942,21.455046,23.301481,25.323546,0.044388,swf


In [22]:
dct_df[reg]['bbp_443_giop']['df_all'].loc[:'2007'].tail()

,bbp_443_giop_adj_mean,phyto_C_2.5%,phyto_C_mean,phyto_C_97.5%,phyto_C_anomaly,sensor
datetime,,,,,,
2007-08-15,0.001967,21.674066,23.594234,25.610215,0.650477,both
2007-09-15,0.001964,21.688976,23.560242,25.619215,-0.003653,both
2007-10-15,0.002034,22.355823,24.374692,26.468022,0.070558,both
2007-11-15,0.002008,22.165679,24.070648,26.207698,-0.215027,both
2007-12-15,0.001968,21.687035,23.608466,25.627197,0.043506,both


In [23]:
dct_df[reg]['bbp_443_giop']['df_all'].loc['2008':].head()

,bbp_443_giop_adj_mean,phyto_C_2.5%,phyto_C_mean,phyto_C_97.5%,phyto_C_anomaly,sensor
datetime,,,,,,
2008-01-15,0.001881,20.831817,22.580714,24.552605,-0.287336,aqua
2008-02-15,0.001914,21.185311,22.972619,24.984008,0.078001,aqua
2008-03-15,0.001921,21.255599,23.054129,25.069227,-0.202964,aqua
2008-04-15,0.001934,21.363554,23.207739,25.216820,-0.093886,aqua
2008-05-15,0.001876,20.778901,22.524139,24.490607,-0.291760,aqua


In [27]:
for reg in regions:
    for prod in prods:
        df_name = f'df_{reg}_{prod}_test_data_consolidated'
        dct_df[reg][prod]['df_all'].to_pickle(f'../PklJar/{df_name}.pkl')

In [28]:
with open('../PklJar/dct_df_test.pkl', 'wb') as f:
    pickle.dump(dct_df, f, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
import numpy as np
import pandas as pd
n = np.arange(10)
p = np.linspace(0, 1)

In [8]:
d = pd.DataFrame([[ni, pi] for ni in n for pi in p], columns=['n', 'p'])

In [16]:
d_ = d.to_dict(orient='list')

In [17]:
d_['n']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
